In [191]:
import torch
from numpy import genfromtxt
import numpy as np
from tqdm import tqdm

data = genfromtxt('/content/data.csv', delimiter=',')

In [192]:
data = data[1:]

In [193]:
data.shape

(2909, 12)

In [194]:
data = np.delete(data, range(0,3) , 1)

In [195]:
rolling = []
newData = []
buf = data[0][8]
for i in data:
    if i[8] != buf:
        buf = i[8]
        rolling = []
    rolling.append(i)
    if len(rolling) == 10:
        newData.append(np.array(rolling))
        rolling.pop()
newData = np.array(newData)

In [196]:
newData.shape

(2891, 10, 9)

In [197]:
data = newData
np.random.shuffle(data)
y = []
for i,x in enumerate(data):
    y.append(x[0][8])
data = np.delete(data, 8, 2)

In [200]:
y = np.array(y)

In [201]:
def convert(n):
    if n == 1:
        return [0.,1.]
    else:
        return [1.,0.]
y = np.asarray(list(map(convert,y)))

In [132]:
np.save("/content/drive/MyDrive/Focusist/y_lstm.npy", y)
np.save("/content/drive/MyDrive/Focusist/x_lstm.npy", data)

In [202]:
y

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [203]:
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
bs = 16
test_size = 500
data, y = map(torch.tensor, (data, y))
data = F.normalize(data, dim=1)
data = data.double()
y = y.double()

train_ds = TensorDataset(data[test_size:,:], y[test_size:])
valid_ds = TensorDataset(data[:test_size,:], y[:test_size])

In [227]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb.double())
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
    return loss.item(), len(xb)
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)
def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
    )

In [225]:
from torch import nn
class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.dropout = nn.Dropout(0.5)
        self.layer1 = nn.Linear(80, 70)
        self.layer2 = nn.Linear(70, 50)
        self.layer3 = nn.Linear(50, 50)
        self.layer4 = nn.Linear(50, 10)
        self.layer5 = nn.Linear(10, 2)
        self.sigmoid = nn.Sigmoid()
    def forward(self, xb):
        initial = xb.size(0)
        xb = xb.view(initial, 1, -1)
        xb = self.layer1(xb)
        xb = self.layer2(xb)
        xb = self.dropout(xb)
        xb = self.layer3(xb)
        xb = self.dropout(xb)
        xb = self.layer4(xb)
        xb = self.dropout(xb)
        xb = self.layer5(xb)
        xb = self.sigmoid(xb.view(initial,-1))
        return xb.double()

In [228]:
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)

In [237]:
from torch import optim
brain = model()
opt = optim.Adam(brain.parameters(), lr=0.002)
loss_func = F.binary_cross_entropy
fit(27, brain.double(), loss_func, opt, train_dl, valid_dl)

0 1.5807145574020992e-06
1 1.1038233584785875e-06
2 7.78755212544099e-07
3 5.710067071663429e-07
4 4.3343558664940346e-07
5 3.3979493298062523e-07
6 2.7119296515148977e-07
7 2.2139069704519811e-07
8 1.8330597844892274e-07
9 1.5422285531843197e-07
10 1.3071517082376903e-07
11 1.1218300774497517e-07
12 9.694591576631564e-08
13 8.435732136573989e-08
14 7.388645668361135e-08
15 6.528928724724503e-08
16 5.760072799240676e-08
17 5.118403448016039e-08
18 4.574256647839299e-08
19 4.084381699033208e-08
20 3.665153109187854e-08
21 3.298739884016982e-08
22 2.977398443158039e-08
23 2.691374902199148e-08
24 2.4398184073338707e-08
25 2.211581091968876e-08
26 2.0130711349013226e-08


In [241]:
wrong = 0
right = 0
for batch in valid_dl:
    x,y = batch
    guesses = brain(x).max(1)[1]
    for i,guess in enumerate(guesses):
        if guess == y.max(1)[1][i]:
            right+=1
        else:
            wrong+=1
print("Right: "+str(right))
print("Wrong: "+str(wrong))

Right: 500
Wrong: 0


In [242]:
torch.save(brain.state_dict(), "brain.pth")